In [4]:
!tree .

.
├── data
│   ├── test
│   │   ├── 1.jpg
│   │   ├── 10.jpg
│   │   ├── 11.jpg
│   │   ├── 2.jpg
│   │   ├── 3.jpg
│   │   ├── 4.jpg
│   │   ├── 5.jpg
│   │   ├── 6.jpg
│   │   ├── 7.jpg
│   │   ├── 8.jpg
│   │   └── 9.jpg
│   └── train
│       ├── cat.0.jpg
│       ├── cat.1.jpg
│       ├── cat.10.jpg
│       ├── cat.2.jpg
│       ├── cat.3.jpg
│       ├── cat.4.jpg
│       ├── cat.5.jpg
│       ├── cat.6.jpg
│       ├── cat.7.jpg
│       ├── cat.8.jpg
│       └── cat.9.jpg
└── main.ipynb

3 directories, 23 files


In [ ]:
import os

import cv2
import numpy as np

minPlateRatio = 0.5  # 车牌最小比例
maxPlateRatio = 6  # 车牌最大比例
# 定义蓝底车牌的hsv颜色区间
# lower_blue = np.array([100, 40, 50])
# higher_blue = np.array([140, 255, 255])
lower_blue = np.array([100, 40, 50])
higher_blue = np.array([140, 255, 255])


# 找到符合车牌形状的矩形
def findPlateNumberRegion(img):
    region = []
    # 查找外框轮廓
    contours_img, contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    print("contours lenth is :%s" % (len(contours)))
    # 筛选面积小的
    list_rate = []
    for i in range(len(contours)):
        cnt = contours[i]
        # 计算轮廓面积
        area = cv2.contourArea(cnt)
        # 面积小的忽略
        if area < 1000:
            continue
        # 转换成对应的矩形（最小）
        rect = cv2.minAreaRect(cnt)
        # print("rect is:%s" % {rect})
        # 根据矩形转成box类型，并int化
        box = np.int32(cv2.boxPoints(rect))
        # 计算高和宽
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
        # 正常情况车牌长高比在2.7-5之间,那种两行的有可能小于2.5，这里不考虑
        ratio = float(width) / float(height)
        rate = getxyRate(cnt)
        print("area", area, "ratio:", ratio, "rate:", rate)
        if ratio > maxPlateRatio or ratio < minPlateRatio:
            continue
        # 符合条件，加入到轮廓集合
        region.append(box)
        list_rate.append(ratio)
    index = getSatifyestBox(list_rate)
    return region[index]

#找出最有可能是车牌的位置
def getSatifyestBox(list_rate):
    for index, key in enumerate(list_rate):
        list_rate[index] = abs(key - 3)
    print(list_rate)
    index = list_rate.index(min(list_rate))
    print(index)
    return index


def getxyRate(cnt):
    x_height = 0
    y_height = 0
    x_list = []
    y_list = []
    for location_value in cnt:
        location = location_value[0]
        x_list.append(location[0])
        y_list.append(location[1])
    x_height = max(x_list) - min(x_list)
    y_height = max(y_list) - min(y_list)
    return x_height * (1.0) / y_height * (1.0)

file = "timg11.jpg"
# def location(file):
img = cv2.imread(file)
# 转换成hsv模式图片
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
cv2.imshow("hsv_img", hsv_img)
cv2.waitKey(0)
# 找到hsv图片下的所有符合蓝底颜色区间的像素点，转换成二值化图像
mask = cv2.inRange(hsv_img, lower_blue, higher_blue)
res = cv2.bitwise_and(img, img, mask=mask)
cv2.imshow("res", res)
cv2.waitKey(0)

# 灰度化
gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
cv2.imshow("gray", gray)
cv2.waitKey(0)
# 高斯模糊：车牌识别中利用高斯模糊将图片平滑化，去除干扰的噪声对后续图像处理的影响
gaussian = cv2.GaussianBlur(gray, (3, 3), 0, 0, cv2.BORDER_DEFAULT)
cv2.imshow("gaussian", gaussian)
cv2.waitKey(0)
#sobel算子：车牌定位的核心算法，水平方向上的边缘检测，检测出车牌区域
sobel = cv2.convertScaleAbs(cv2.Sobel(gaussian, cv2.CV_16S, 1, 0, ksize=3))
cv2.imshow("sobel", sobel)
cv2.waitKey(0)
#进一步对图像进行处理，强化目标区域，弱化背景。
ret, binary = cv2.threshold(sobel, 150, 255, cv2.THRESH_BINARY)
cv2.imshow("binary", binary)
cv2.waitKey(0)



# 进行闭操作，闭操作可以将目标区域连成一个整体，便于后续轮廓的提取
element = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 5))
closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, element)
# 进行开操作，去除细小噪点
# eroded = cv2.erode(closed, None, iterations=1)
# dilation = cv2.dilate(eroded, None, iterations=1)
cv2.imshow("closed", closed)
cv2.waitKey(0)


# 查找并筛选符合条件的矩形区域
region = findPlateNumberRegion(closed)
cv2.drawContours(img, [region], 0, (0, 255, 0), 2)
cv2.imshow("img", img)
cv2.waitKey(0)


# if __name__ == '__main__':
# path = "car_id"
# fileList = os.listdir(path)
# for file in fileList:
#     try:
#         location(path + "/" + file)
#     except:
#         print("异常:",file)


# location(file)

In [7]:
!ls -l 

total 4608
drwxr-xr-x  5 liufang  staff      160  4 17 15:24 data
-rw-r--r--  1 liufang  staff     8263  4 17 19:46 main.ipynb
-rw-r--r--@ 1 liufang  staff  2344028  4 17 19:44 timg11.jpg


In [ ]:
# coding=gbk
"""
车牌定位模块，尝试将Sobel边缘和基于颜色HSV的两种定位方法互相筛选，实现车牌定位
__author__ = 'kuang'
2019.04.20   7号宿舍楼
"""

import os
import time
import cv2 as cv
import numpy as np

#找出最有可能是车牌的位置
def getSatifyestBox(list_rate):
    for index, key in enumerate(list_rate):
        list_rate[index] = abs(key - 3)
    index = list_rate.index(min(list_rate))   #index函数作用是：若list_rate中存在index括号中的内容，则返回括号内字符串的索引值
    return index

def location(img):
    #读取图片并统一尺寸
    img_resize = cv.resize(img,(640,480),)
    #高斯模糊+中值滤波
    img_gaus = cv.GaussianBlur(img_resize,(5,5),0)          #高斯模糊
    img_med = cv.medianBlur(img_gaus,5)                     #中值滤波

    #HSV模型处理，直至二值化
    #转换为HSV模型
    img_hsv = cv.cvtColor(img_med,cv.COLOR_BGR2HSV)        #hsv模型
    lower_blue = np.array([100,40,50])
    higher_blue = np.array([140,255,255])
    mask = cv.inRange(img_hsv,lower_blue,higher_blue)    #掩膜操作
    img_res = cv.bitwise_and(img_med,img_med,mask=mask)

    #灰度化+二值化
    img_gray_h = cv.cvtColor(img_res,cv.COLOR_BGR2GRAY)    #转换了灰度化
    ret1,img_thre_h = cv.threshold(img_gray_h,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

    #进行Sobel算子运算，直至二值化
    img_gray_s = cv.cvtColor(img_med,cv.COLOR_BGR2GRAY) 

    #sobel算子运算
    img_sobel_x = cv.Sobel(img_gray_s,cv.CV_32F,1,0,ksize=3)    #x轴Sobel运算
    img_sobel_y = cv.Sobel(img_gray_s,cv.CV_32F,0,1,ksize=3)
    img_ab_y = np.uint8(np.absolute(img_sobel_y))
    img_ab_x = np.uint8(np.absolute(img_sobel_x))               #像素点取绝对值
    img_ab = cv.addWeighted(img_ab_x, 0.5, img_ab_y, 0.5,0)   #将两幅图像叠加在一起（按一定权值）
    #考虑再加一次高斯去噪
    img_gaus_1 = cv.GaussianBlur(img_ab,(5,5),0)          #高斯模糊

    #二值化操作
    ret2,img_thre_s = cv.threshold(img_gaus_1,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)     #正二值化
	
	#颜色空间与边缘算子的图像互相筛选
    #同时遍历两幅二值图片，若两者均为255，则置255
    img_1 = np.zeros(img_thre_h.shape,np.uint8)  #重新拷贝图片
    height = img_resize.shape[0]    #行数
    width = img_resize.shape[1]     #列数
    for i in range(height):
        for j in range(width):
            h = img_thre_h[i][j]
            s = img_thre_s[i][j]
            if h ==255 and s ==255 :
                img_1[i][j] = 255
            else:
                img_1[i][j] = 0
    # cv.imshow('threshold',img_1)
    # cv.waitKey(0)

    #二值化后的图像进行闭操作
    kernel = np.ones((14,18),np.uint8)                           
    img_close = cv.morphologyEx(img_1,cv.MORPH_CLOSE,kernel)    #闭操作
    img_med_2 = cv.medianBlur(img_close,5)
    # cv.imshow('close',img_med_2)
    # cv.waitKey(0)

    #查找轮廓
    regions = []         #区域
    list_rate = []
    img_input = img_med_2.copy()
    contours,hierarchy = cv.findContours(img_input,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    #   筛选面积最小的
    for contour in contours:
        #计算该轮廓的面积
        area = cv.contourArea(contour)
        #面积小的都筛选掉
        if area < 2000:
            continue
        #轮廓近似,epsilon，是从轮廓到近似轮廓的最大距离。是一个准确率参数，好的epsilon的选择可以得到正确的输出。True决定曲线是否闭合。
        epslion = 1e-3 * cv.arcLength(contour,True)
        approx = cv.approxPolyDP(contour,epslion,True)            #曲线折线化
        #找到最小的矩形，该矩形可能有方向
        rect = cv.minAreaRect(contour)
        #box是四个点的坐标
        box = cv.boxPoints(rect)
        box = np.int0(box)
        #计算高和宽
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
        #车牌正常情况下长高比为2-5之间（精确一点可为（2.2,3.6））
        ratio = float(width) / float(height)
        if ratio > 2 and ratio < 5:
            regions.append(box)
            list_rate.append(ratio)
    #输出车牌的轮廓
    print('[INF0]:Detect %d license plates' % len(regions))    #输出疑似车牌图块的数量
    index = getSatifyestBox(list_rate)
    region = regions[index]
    #用绿线画出这些找到的轮廓
    #重新申请空间拷贝，因为drawcontours会改变原图片
    img_2 = np.zeros(img_resize.shape,np.uint8)
    img_2 = img_resize.copy()
    cv.drawContours(img_2, [region], 0, (0, 255, 0), 2)
    # cv.imshow('result',img_2)
    # cv.waitKey(0)

    #定位后需对车牌图像做后面的字符分割等处理，因此需要将车牌图块单独截取出来，截取轮廓
    Xs = [i[0] for i in region]
    YS = [i[1] for i in region]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(YS)
    y2 = max(YS)
    height_1 = y2 - y1
    width_1 = x2 - x1
    img_crop = img_resize[y1:y1+height_1,x1:x1+width_1]
    # cv.imshow('resultcut',img_crop)
    # cv.waitKey(0)
	
	
	
	#后面是自己的一些想法，希望能够对截取到的车牌图块再细致处理一下，使其仅保留车牌部分，但作用貌似也不大（苦笑）
    #假设再进行一次HSV
    img_hsv_1 = cv.cvtColor(img_crop,cv.COLOR_BGR2HSV)        #hsv模型
    lower_blue_1 = np.array([100,90,90])
    higher_blue_1 = np.array([140,255,255])
    mask_1 = cv.inRange(img_hsv_1,lower_blue_1,higher_blue_1)    #掩膜操作
    img_res_1 = cv.bitwise_and(img_crop,img_crop,mask=mask_1)
    
    #灰度化+二值化
    img_gray_1 = cv.cvtColor(img_res_1,cv.COLOR_BGR2GRAY)    #转换了灰度化
    ret3,img_thre_1 = cv.threshold(img_gray_1,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    
    height_2 = img_thre_1.shape[0]      #获得行数
    width_2 = img_thre_1.shape[1]       #获得列数
    white_min = []
    white_max = []
    a = 0
    b = 0
    #从每行开始遍历，记录每行第一个及最后一个白色像素点的列数
    for i in range(height_2):
        for j in range(width_2):
            h = img_thre_1[i,j]
            if h == 255:
                a = j
                white_min.append(a)
                break
    a = min(white_min)
    for q in range(height_2-1,-1,-1):
        for w in range(width_2-1,-1,-1):
            ps = img_thre_1[q,w]
            if ps == 255:
                b = w
                white_max.append(b)
                break
    b = max(white_max)
    white_min1 = []
    white_max1 = []
    c = 0
    d = 0
    #从每一列开始遍历，记录每一行第一个白色像素点及最后一个像素点的行数
    for i in range(width_2):
        for j in range(height_2):
            h = img_thre_1[j,i]
            if h == 255:
                c = j
                white_max1.append(c)
                break
    c = min(white_max1)
    for q in range(width_2-1,-1,-1):
        for w in range(height_2-1,-1,-1):
            ps = img_thre_1[w,q]
            if ps == 255:
                d = w
                white_min1.append(d)
                break
    d = max(white_min1)
    img_cut = img_crop[c:d,a:b]
    # cv.imshow('final',img_cut)
    # cv.waitKey(0)
    return img_cut

# if __name__ == "__main__":
img = cv.imread("timg11.jpg")     #输入自己的图片
cv.imshow("plate",img)
cv.waitKey(0)
img_p = location(img)
cv.imshow("plate",img_p)
cv.waitKey(0)



In [4]:
# -*- coding: utf-8 -*-
import os
import numpy as np

import sys
import cv2
import numpy
import time

def cut_image(img_in):
    # img_in = cv2.imread('./car_image/letters.png')
    gray = cv2.cvtColor(img_in, cv2.COLOR_BGR2GRAY)
    # cv2.imwrite('./test_ans/2.1.gray.jpg', gray)
    ret, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    cv2.imwrite("./test_ans/binary_2.jpg", binary)
    # cv2.imshow("binary", binary)
    # cv2.waitKey(0)

    root = './test_ans/'
    dsize = 28
    data = np.array(binary)
    len_x = data.shape[0]
    len_y = data.shape[1]
    min_val = 10  # 设置最小的文字像素高度，防止切分噪音字符

    start_i = -1
    end_i = -1
    rowPairs = []  # 存放每行的起止坐标

    # 行分割
    for i in range(len_x):
        if (data[i].any() and start_i < 0):
            start_i = i
        elif (data[i].any()):
            end_i = i
        elif (not data[i].any() and start_i >= 0):
            if (end_i - start_i >= min_val):
                rowPairs.append((start_i, end_i))
            else:
                start_i, end_i = -1, -1
    if start_i>=0 & len(rowPairs)==0:
        rowPairs.append((start_i, len_x))

    # 列分割
    start_j = -1
    end_j = -1
    min_val_word = 5  # 最小文字像素长度
    number = 0  # 分割后保存编号

    now_time = int(time.time())
    for start, end in rowPairs:
        for j in range(len_y):
            if (data[start: end, j].any() and start_j < 0):
                start_j = j
            elif (not data[start: end, j].all() and start_j >= 0):
                end_j = j

            if (not data[start: end, j].any() and start_j >= 0):
                if (end_j - start_j >= min_val_word):
                    # print end_j - start_j, end-start
                    if start_j-2>=0 and end_j+2<=len_y:
                        tmp = data[start:end, start_j-2: end_j+2]
                    else:
                        tmp = data[start:end, start_j: end_j]
                    im2save = cv2.resize(tmp, (dsize, dsize))  # 归一化处理
                    number += 1

                    cv2.imwrite(root + '%s_%d.bmp' % (now_time, number), im2save)
                    print("saved successfully!")

                start_j, end_j = -1, -1
    if number>=5:
        print('cut one image successfully!')

def preprocess(gray):
    # cv2.imshow('gray', gray)
    # 高斯平滑
    gaussian = cv2.GaussianBlur(gray, (3, 3), 0, 0, cv2.BORDER_DEFAULT)
    # 中值滤波
    median = cv2.medianBlur(gaussian, 5)
    # Sobel算子，X方向求梯度
    sobel = cv2.Sobel(median, cv2.CV_8U, 1, 0)

    cv2.imwrite('./test_ans/1.1.gray.jpg', gray)
    cv2.imwrite('./test_ans/1.2.sobel.jpg', sobel)
    # cv2.imshow('sobel', sobel)

    # 二值化
    ret, binary = cv2.threshold(sobel, 170, 255, cv2.THRESH_BINARY)
    cv2.imwrite('./test_ans/1.3.binary.jpg', binary)

    # 膨胀和腐蚀操作的核函数
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (35, 1))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (36, 5))
    # 膨胀一次，让轮廓突出
    dilation = cv2.dilate(binary, element1, iterations=2)
    cv2.imwrite('./test_ans/1.3.dilate.jpg', dilation)

    # 腐蚀一次，去掉细节
    erosion = cv2.erode(dilation, element1, iterations=4)
    cv2.imwrite('./test_ans/1.4.close.jpg', erosion)

    # 再次膨胀，让轮廓明显一些
    dilation2 = cv2.dilate(erosion, element2, iterations=2)
    cv2.imwrite('./test_ans/1.5.open.jpg', dilation2)

    # erosion2 = cv2.erode(dilation2, element2, iterations=4)
    # dilation3 = cv2.dilate(erosion2, element2, iterations=2)

    # cv2.imshow('dilation2', dilation2)
    # cv2.waitKey(0)
    return dilation2

def findPlateNumberRegion(img):
    region = []
    # 查找轮廓
    _, contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # 筛选面积小的
    for i in range(len(contours)):
        cnt = contours[i]
        # 计算该轮廓的面积
        area = cv2.contourArea(cnt)

        # 面积小的都筛选掉
        if (area < 5000):
            continue

        # 轮廓近似，作用很小
        epsilon = 0.001 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        # 找到最小的矩形，该矩形可能有方向
        rect = cv2.minAreaRect(cnt)
        print "rect is: "
        print rect

        # box是四个点的坐标
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        # 计算高和宽
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
        # 车牌正常情况下长高比在2.7-5之间
        ratio = float(width) / float(height)
        print ratio
        # if (ratio > 5 or ratio < 2):
        #     continue
        region.append(box)

    return region


def detect(img):
    # 转化成灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 形态学变换的预处理
    dilation = preprocess(gray)

    # 查找车牌区域
    region = findPlateNumberRegion(dilation)
    # 用绿线画出这些找到的轮廓
    if len(region)>0:
        print 'find area '
    else:
        print 'not find area and returned'
    for box in region:
        ys = [box[0, 1], box[1, 1], box[2, 1], box[3, 1]]
        xs = [box[0, 0], box[1, 0], box[2, 0], box[3, 0]]
        ys_sorted_index = np.argsort(ys)
        xs_sorted_index = np.argsort(xs)

        x1 = box[xs_sorted_index[0], 0]
        x2 = box[xs_sorted_index[3], 0]

        y1 = box[ys_sorted_index[0], 1]
        y2 = box[ys_sorted_index[3], 1]

        img_org2 = img.copy()
        img_plate = img_org2[y1:y2, x1:x2]
        cv2.imwrite('./test_ans/number_plate_%s.jpg'%int(time.time()), img_plate)
        cv2.drawContours(img, [box], 0, (0, 255, 0), 2)

        # cv2.imshow('plate', img)
        cut_image(img_plate)

        print 'write number_plate.jpg successfully'
    # cv2.namedWindow('img', cv2.WINDOW_NORMAL)

    # 带轮廓的图片
    cv2.imwrite('./test_ans/contours_%s.png'%int(time.time()), img)
    # cv2.imshow('l', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# if __name__ == '__main__':
    # imagePath = './car_image/car3.jpg'
imagePath = '/Users/liufang/Desktop/src/car_image/car3.jpg'
img = cv2.imread(imagePath)
detect(img)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("rect is: ")? (<ipython-input-4-d64ce6daf253>, line 135)

In [2]:
    !ls -l /Users/liufang/Desktop/src/car_image/car3.jpg


total 120
drwxr-xr-x  21 liufang  staff    672  4 17 21:37 car_image
-rw-r--r--@  1 liufang  staff   6312  4 18 10:49 detect.py
-rw-r--r--@  1 liufang  staff    141  4 17 21:37 say.py
-rw-r--r--@  1 liufang  staff    156  4 17 21:37 shell.sh
-rw-r--r--@  1 liufang  staff     98  4 17 21:37 test.sh
drwxr-xr-x   3 liufang  staff     96  4 17 22:56 test_ans
-rw-r--r--@  1 liufang  staff  12208  4 17 21:38 train-license-digits.py
-rw-r--r--@  1 liufang  staff  12237  4 17 21:38 train-license-letters.py
-rw-r--r--@  1 liufang  staff  12009  4 17 21:38 train-license-province.py
drwxr-xr-x   5 liufang  staff    160  4 17 21:49 train-saver
-rw-r--r--@  1 liufang  staff    126  4 17 21:49 train.sh
drwxr-xr-x   3 liufang  staff     96  4 17 21:49 train_images
